In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import glob     
import json
import os
import shutil

#########################################################################
# 어플에서 올라온 이미지 처음 경로 - 초기 설정 필요합니다.
application_image = r'C:\Users\sj990\MachineLearning\CAPSTONE_DESIGN_AI_Module\yolo\first_image_folder - 복사본'
#########################################################################

image_directory = application_image
# jpg, png, jpeg 형식 (리스트형식으로 저장된다, 경로에는 파일한장만 존재하게)
image_files = glob.glob(image_directory + '/*.png') + glob.glob(image_directory + '/*.jpg') + glob.glob(image_directory + '/*.jpeg')
# 리스트중 첫번째것만 사용
first_image = image_files[0]

# 객체검출 실행
!python yolov5/detect.py --weights best_noodle_drink.pt --img 640 --conf 0.65 --source "{first_image}" --project ../yolo --save-crop

# 검출경로
crops_path = os.path.join('exp', 'crops')


###############################################################
# 객체 폴더 있는지 탐지 (매번 추가) - 작업중
noodle_detect = os.path.exists(os.path.join(crops_path, 'noodle'))
drink_detect = os.path.exists(os.path.join(crops_path, 'drink'))

if noodle_detect:
    yolo_img = os.path.join(crops_path, 'noodle')
    classification_label = "noodle"
    
elif drink_detect:
    yolo_img = os.path.join(crops_path, 'drink')
    classification_label = "drink"
    
    
else:
    print("객체검출실패")
###############################################################
    
    
yolo_image_directory = yolo_img
# jpg, png, jpeg 형식 (리스트형식으로 저장된다, 경로에는 파일한장만 존재하게)
yolo_image_files = glob.glob(yolo_image_directory + '/*.png') + glob.glob(yolo_image_directory + '/*.jpg') + glob.glob(yolo_image_directory + '/*.jpeg')
# 리스트중 첫번째것만 사용
yolo_image_path = yolo_image_files[0]

    
class Model:
    # 초기화
    def __init__(self, image_path, model_path, class_labels_path):
        self.image_path = image_path
        self.model_path = model_path
        self.class_labels_path = class_labels_path
        
    #예측 (전처리된이미지와 클래스라벨을 삽입)
    def preds_model(self, img, class_labels):
        model = tf.keras.models.load_model(self.model_path,  custom_objects={'KerasLayer': hub.KerasLayer})
        preds = model.predict(img) 
        
        # 결과값 상위 1개
        top_preds_idx = preds[0].argsort()[ : : -1][ : 1]
        top_preds_labels = [class_labels[idx] for idx in top_preds_idx]
        top_preds_probs = preds[0][top_preds_idx]
        return top_preds_labels, top_preds_probs
    
    #이미지전처리_1 (기본)
    def img_pre_default(self):  
        img = tf.keras.preprocessing.image.load_img(self.image_path, target_size=(256, 256))
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = img.astype('float32') / 255.0
        img = np.expand_dims(img, axis=0)
        return img

    #라벨불러오기
    def class_label(self):
        class_labels = []
        with open(self.class_labels_path, 'r') as file:
            for line in file:
                class_labels.append(line.strip())
        return class_labels
    
    #결과출력
    def print_result(self, top_preds_labels, top_preds_probs):
        results = []
        #json형식 results
        for label, prob in zip(top_preds_labels, top_preds_probs):
            result = {
                '상품': label,
                '정확도': round(float(prob), 2)
            }
            results.append(result)
        print(results)
        return results
  

#########################################################################
# 저장한 모델, 라벨들 경로 - 초기 설정 필요합니다.

# 면류 모델경로, 면류 라벨경로
image_path = yolo_image_path
noodle_model_path = r'C:\Users\sj990\Desktop\종설\모델,라벨\모델\model_noodle.h5'
noodle_label_path = r'C:\Users\sj990\Desktop\종설\모델,라벨\라벨\model_noodle_label.txt'

# 음료 모델경로, 음료 라벨경로
drink_model_path = r'C:\Users\sj990\Desktop\종설\모델,라벨\라벨\model_drink.h5'
drink_label_path = r'C:\Users\sj990\Desktop\종설\모델,라벨\라벨\model_drink_label.txt'


# 미분류 모델경로, 면류 라벨경로
unclassified_model_path = r'C:\Users\sj990\Desktop\종설\모델,라벨\모델\model_unclass.h5'
unclassified_label_path = r'C:\Users\sj990\Desktop\종설\모델,라벨\라벨\model_unclass_label.txt'


################################경로설정################################



### 특징분류 ###
if classification_label == "noodle":
    #모델정의
    model_noodle = Model(image_path, noodle_model_path, noodle_label_path)
    #예측
    result_noodle_labels, result_noodle_probs = model_noodle.preds_model(model_noodle.img_pre_default(), model_noodle.class_label())
    #결과출력
    noodle_result = model_noodle.print_result(result_noodle_labels, result_noodle_probs)
    
    
    # json파일 저장필요할시 주석 제거후 사용
    with open('results.json', 'w', encoding='utf-8') as file:
        json.dump(noodle_result, file, ensure_ascii=False)
        
elif classification_labe =="drink":
    #모델정의
    model_drink = Model(image_path, drink_model_path, drink_label_path)
    #예측
    result_drink_labels, result_drink_probs = model_drink.preds_model(model_drink.img_pre_default(), model_drink.class_label())
    #결과출력
    drink_result = model_drink.print_result(result_drink_labels, result_drink_probs)
    
    # json파일 저장필요할시 주석 제거후 사용
    with open('results.json', 'w', encoding='utf-8') as file:
        json.dump(drink_result, file, ensure_ascii=False)
        
    
elif classification_labe =="미분류":
    #모델정의
    model_unclassified = Model(image_path, unclassified_model_path, unclassified_label_path)
    #예측
    result_unclassified_labels, result_unclassified_probs = model_unclassified.preds_model(model_unclassified.img_pre_default(), model_unclassified.class_label())
    #결과출력
    unclassified_result = model_unclassified.print_result(result_unclassified_labels, result_unclassified_probs)
    
    
    # json파일 저장필요할시 주석 제거후 사용
    with open('results.json', 'w', encoding='utf-8') as file:
        json.dump(unclassified_result, file, ensure_ascii=False)
    
    
# 작업끝난후 yolo 이미지 폴더 삭제
shutil.rmtree("exp")

# 작업끝난후 어플리케이션에서 올라온 폴더 삭제
shutil.rmtree(application_image)

detect: weights=['best_noodle_drink.pt'], source=C:\Users\sj990\MachineLearning\CAPSTONE_DESIGN_AI_Module\yolo\first_image_folder - \1.jpg, data=yolov5\data\coco128.yaml, imgsz=[640, 640], conf_thres=0.65, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=../yolo, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-198-g34c2187 Python-3.9.7 torch-2.0.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 C:\Users\sj990\MachineLearning\CAPSTONE_DESIGN_AI_Module\yolo\first_image_folder - \1.jpg: 640x480 1 noodle, 118.1ms
Speed: 1.0ms pre-process, 118.1ms inference, 0.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to ..\yolo\exp


[{'상품': '10111_오뚜기컵누들매콤37.8G', '정확도': 0.99}]
